In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

In [1]:
import ray

ray.shutdown()  # Ensure any previous Ray instance is stopped
ray.init(ignore_reinit_error=True, include_dashboard=True, dashboard_port=8265)

#print(f"Dashboard is running at {ray.get_dashboard_url()}")


/opt/anaconda3/envs/neuromancer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-27 13:36:04,823	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-02-27 13:36:04,846	INFO worker.py:1654 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2025-02-27 13:36:04,851	INFO worker.py:1832 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:9000 


Python version:,3.10.15
Ray version:,2.42.1
Dashboard:,http://127.0.0.1:9000


In [2]:
import torch
from neuromancer.library import *
from neuromancer.sindy import *
from neuromancer import psl
from torch.utils.data import DataLoader
from matplotlib.lines import Line2D
from neuromancer.system import Node, System
from neuromancer.dynamics import integrators
from neuromancer.trainer import Trainer
from neuromancer.problem import Problem
from neuromancer.loggers import BasicLogger
from neuromancer.dataset import DictDataset
from neuromancer.constraint import variable
from neuromancer.loss import PenaltyLoss
from neuromancer.modules import blocks
from neuromancer.plot import pltOL
import matplotlib.pyplot as plt
import itertools
from ray import train, tune
torch.manual_seed(0)
# For now, we use CPU till we fix the cuda utilization error
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")


2025-02-27 13:36:16,752	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
gt_model_name = "LinearReno_ROM40"
gt_model = psl.systems[gt_model_name]()

ts = gt_model.ts
nx = gt_model.nx
ny = gt_model.ny
nu = gt_model.nu
nd = gt_model.nd


### Getting Data

In [4]:
def normalize(x, mean, std):
    return (x - mean) / std

def get_data(sys, nsim, nsteps, ts, bs):
    """
    :param nsteps: (int) Number of timesteps for each batch of training data
    :param sys: (psl.system)
    :param ts: (float) step size
    :param bs: (int) batch size

    """
    train_sim, dev_sim, test_sim = [sys.simulate(nsim=nsim, ts=ts) for i in range(3)]
    nx = sys.nx
    nu = sys.nu
    nd = sys.nd
    ny = sys.ny
    nbatch = nsim//nsteps
    length = (nsim//nsteps) * nsteps

    mean_x = gt_model.stats['X']['mean']
    std_x = gt_model.stats['X']['std']
    mean_y = gt_model.stats['Y']['mean']
    std_y = gt_model.stats['Y']['std']
    mean_u = gt_model.stats['U']['mean']
    std_u = gt_model.stats['U']['std']
    mean_d = gt_model.stats['D']['mean']
    std_d = gt_model.stats['D']['std']

    trainX = normalize(train_sim['X'][:length], mean_x, std_x)
    trainX = trainX.reshape(nbatch, nsteps, nx)
    trainX = torch.tensor(trainX, dtype=torch.float32)
    trainY = normalize(train_sim['Y'][:length], mean_y, std_y)
    trainY = trainY.reshape(nbatch, nsteps, ny)
    trainY = torch.tensor(trainY, dtype=torch.float32)
    trainU = normalize(train_sim['U'][:length], mean_u, std_u)
    trainU = trainU.reshape(nbatch, nsteps, nu)
    trainU = torch.tensor(trainU, dtype=torch.float32)
    trainD = normalize(train_sim['D'][:length], mean_d, std_d)
    trainD = trainD.reshape(nbatch, nsteps, nd)
    trainD = torch.tensor(trainD, dtype=torch.float32)
    train_data = DictDataset({'X': trainX, 'yn': trainY[:, 0:1, :],
                              'Y': trainY,
                              'U': trainU,
                              'D': trainD}, name='train')
    train_loader = DataLoader(train_data, batch_size=bs,
                              collate_fn=train_data.collate_fn, shuffle=True)

    devX = normalize(dev_sim['X'][:length], mean_x, std_x)
    devX = devX.reshape(nbatch, nsteps, nx)
    devX = torch.tensor(devX, dtype=torch.float32)
    devY = normalize(dev_sim['Y'][:length], mean_y, std_y)
    devY = devY.reshape(nbatch, nsteps, ny)
    devY = torch.tensor(devY, dtype=torch.float32)
    devU = normalize(dev_sim['U'][:length], mean_u, std_u)
    devU = devU[:length].reshape(nbatch, nsteps, nu)
    devU = torch.tensor(devU, dtype=torch.float32)
    devD = normalize(dev_sim['D'][:length], mean_d, std_d)
    devD = devD[:length].reshape(nbatch, nsteps, nd)
    devD = torch.tensor(devD, dtype=torch.float32)
    dev_data = DictDataset({'X': devX, 'yn': devY[:, 0:1, :],
                            'Y': devY,
                            'U': devU,
                            'D': devD}, name='dev')
    dev_loader = DataLoader(dev_data, batch_size=bs,
                            collate_fn=dev_data.collate_fn, shuffle=True)

    testX = normalize(test_sim['X'][:length], mean_x, std_x)
    testX = testX.reshape(1, nbatch*nsteps, nx)
    testX = torch.tensor(testX, dtype=torch.float32)
    testY = normalize(test_sim['Y'][:length], mean_y, std_y)
    testY = testY.reshape(1, nbatch*nsteps, ny)
    testY = torch.tensor(testY, dtype=torch.float32)
    testU = normalize(test_sim['U'][:length], mean_u, std_u)
    testU = testU.reshape(1, nbatch * nsteps, nu)
    testU = torch.tensor(testU, dtype=torch.float32)
    testD = normalize(test_sim['D'][:length], mean_d, std_d)
    testD = testD.reshape(1, nbatch*nsteps, nd)
    testD = torch.tensor(testD, dtype=torch.float32)
    test_data = {'X': testX, 'yn': testY[:, 0:1, :],
                 'Y': testY, 'U': testU, 'D': testD,
                 'name': 'test'}

    return train_loader, dev_loader, test_data

In [5]:
nsim = 1000
nsteps = 20
bs = 100
train_loader, dev_loader, test_data = get_data(gt_model, nsim, nsteps, ts, bs)

In [6]:

def train_func(optim, system, config):
    nsteps = config["n_steps"]

    y = variable("Y")
    yhat = variable('yn')[:,:-1,:]

    # trajectory tracking loss
    reference_loss = config['ref_loss_coef']*(yhat == y)^2
    reference_loss.name = "ref_loss"

    # one-step tracking loss
    onestep_loss = 1.*(yhat[:, 1, :] == y[:, 1, :])^2
    onestep_loss.name = "onestep_loss"

    constraints = []
    l1 = variable([y], lambda y: torch.norm(list(system.parameters())[0]))
    loss_l1 = config["lambda"]*(l1 == 0)
    objectives = [reference_loss, onestep_loss, loss_l1]

    components = [system]
    # create constrained optimization loss
    loss = PenaltyLoss(objectives, constraints)
    # construct constrained optimization problem
    problem = Problem(components, loss)
    trainer = Trainer(
        problem,
        None,
        None,
        optimizer=optim,
        epochs=2500,
        train_metric='train_loss',
        eval_metric='dev_loss',
        warmup=300,
        patience=200,
        epoch_verbose=200
    )
    for _ in range(config['n_iter']):
        train_loader, dev_loader, _ = get_data(gt_model, 1000, nsteps, ts, bs=100)

        trainer.train_data, trainer.dev_data = train_loader, dev_loader
        trainer.problem = problem
        # Train control policy
        best_model = trainer.train()

        # load best trained model
        trainer.model.load_state_dict(best_model)
        nsteps *= 2
        system.nsteps = nsteps
        trainer.badcount = 0

def test(system, test_data):
    nsteps = 300
    system.nsteps = nsteps

    # perform closed-loop simulation
    trajectories_sindy = system(test_data)

    return torch.nn.functional.mse_loss(trajectories_sindy['yn'][:,:-1,:], test_data['Y'][:,:nsteps,:]).item()

In [15]:
ntests = 5
def objective(config):
    torch.manual_seed(0)
    n_steps = config["n_steps"]
    max_freq = config["max_freq"]
    max_degree = config["max_degree"]

    losses = []
    for _ in range(ntests):
        theta_1 = PolynomialLibrary(ny, nu+nd, max_degree=max_degree)
        theta_2 = FourierLibrary(ny, nu+nd, max_freq=max_freq)

        fx = SINDy(CombinedLibrary([theta_2,theta_1]), n_out=ny)
        integrator = integrators.Euler(fx, h=ts)
        combined_ud = Node(lambda u, d: torch.cat([u, d], dim=-1),
                      ['U', 'D'], ['UD'], name="concat_u_d")

        integrator_node = Node(integrator, ['yn', 'UD'], ['y'], name="Sindy")
        y_bound = Node(lambda x: torch.clamp(x, -20., 20.), ['y'], ['yn'], "y_clamp")
        dynamics_model = System([combined_ud, integrator_node, y_bound], name="dynamics_model")

        optimizer = torch.optim.AdamW(dynamics_model.parameters(), lr=0.005)

        config["train"](optimizer, dynamics_model, config)
        losses.append(test(dynamics_model, test_data))  # Compute test accuracy

    losses = torch.tensor(losses)
    tune.report({"eval_loss": torch.mean(losses).item(),
             "variance": torch.var(losses).item(),
             "params": list(dynamics_model.parameters())[0].detach().cpu().numpy().tolist()})




search_space = {"n_steps": tune.grid_search([2,4,8,16]), "n_iter": tune.grid_search([1,2,3,4]),
               "max_degree": tune.grid_search([1,2,3]), "max_freq": tune.grid_search([1,2,3]),
               "train": tune.grid_search([train_func]),
               "lambda": tune.grid_search([1e-1, 1e-3, 1e-5, 1e-7]),
                'ref_loss_coef': tune.grid_search([1., 3., 5., 10.])}

tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="eval_loss",
        mode="min",
    ),
    param_space=search_space,
)
results = tuner.fit()
print("Best config is:", results.get_best_result().config)
#results.get_dataframe().to_csv("csv/tt.csv")

[*** LOG ERROR #0001 ***] [2025-02-27 13:43:08] [ray_log_sink] Failed flush to file /tmp/ray/session_2025-02-27_13-16-56_683176_10494/logs/python-core-driver-05000000ffffffffffffffffffffffffffffffffffffffffffffffff_10835.log: No space left on device
2025-02-27 13:43:08,244	ERROR tune_controller.py:1331 -- Trial task failed for trial objective_2e378_00008
Traceback (most recent call last):
  File "/opt/anaconda3/envs/neuromancer/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/opt/anaconda3/envs/neuromancer/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/opt/anaconda3/envs/neuromancer/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/opt/anaconda3/envs/neuromancer/lib/python3.10/site-packages/ray/_private/worker.py", line 2772, in get
  

OSError: [Errno 28] No space left on device: '/tmp/ray/session_2025-02-27_13-16-56_683176_10494/artifacts/2025-02-27_13-39-30/objective_2025-02-27_13-39-30/driver_artifacts/objective_2e378_00008_8_lambda=0.1000,max_degree=3,max_freq=1,n_iter=1,n_steps=2,ref_loss_coef=1.0000,train=ref_ph_e8d617a4_2025-02-27_13-39-31/error.pkl'